In [1]:
import sys
sys.path.append("../src")
import climber_simulator as cs
import pandas as pd

In [2]:
casual = []
skilled = []
elite = []

# build 100 data for each category
for _ in range(100):
    casual.append(cs.simulator('casual'))
    skilled.append(cs.simulator('skilled'))
    elite.append(cs.simulator('elite'))

df_casual = pd.DataFrame(casual)
df_skilled = pd.DataFrame(skilled)
df_elite = pd.DataFrame(elite)


In [3]:
df_casual.to_csv("casual_dataset.csv",index=False)
df_skilled.to_csv("skilled_dataset.csv",index=False)
df_elite.to_csv("elite_dataset.csv",index=False)

# Label the feasibility to climbers with differnt climbing routes

In [4]:
from ast import literal_eval

casual_climbers = pd.read_csv("casual_dataset.csv")
skilled_climbers = pd.read_csv("skilled_dataset.csv")
elite_climbers = pd.read_csv("elite_dataset.csv")
routes = pd.read_csv("simulated_routes.csv")
routes["parsed_points"] = routes["points"].apply(literal_eval)

In [5]:
from assign_feasibility import is_route_feasible
from math import dist

casuals = []
skilleds = []
elites = []

category = {
    "casual_climbers": casual_climbers,
    "skilled_climbers": skilled_climbers,
    "elite_climbers": elite_climbers
}

# Label feasibility
for name, type in category.items():
    for i, climber in type.iterrows():
        for j, row in routes.iterrows():
            points = row['parsed_points']
            max_distance = max(dist(points[k], points[k + 1]) for k in range(len(points) - 1))
            sample = {
                "climber_id": f"climber_{i:03d}",
                "route_id": row["route_id"],
                "feasible": int(is_route_feasible(climber, points)),
                "features": {
                    "height": round(climber["height"], 1),
                    "weight": round(climber["weight"], 1),
                    "ape_index": round(climber["ape_index"], 2),
                    "strength": int(climber["strength"]),
                    "flexibility": int(climber["flexibility"]),
                    "n_points": len(points),
                    "max_hold_distance": round(max_distance, 2)
                }
            }
            if name == "casual_climbers":
                casuals.append(sample)
            elif name == "skilled_climbers":
                skilleds.append(sample)
            else:
                elites.append(sample)

In [6]:
from route_parser import pixel_dist

for j, row in routes.iterrows():
    points = row['parsed_points']
    max_distance = max(pixel_dist(points[k], points[k + 1]) for k in range(len(points) - 1))
    print(max_distance)

263.3359607516603
243.3781192322061
266.71519388552593
260.4782795894311
244.6262573923436
206.59672373455305
219.5507555384169
209.15431377150884
219.5507555384169
219.5507555384169
229.48368030210779
245.92959245996062
231.97405635558985
229.1793446190123
267.81650441866094
198.11815585165647
264.45954037266233
211.40951695090962
223.5285095354683
267.73110368372625
263.0761025495343
139.94076259480536
255.1785420453847
269.33128516164663
261.85185296270106
264.28502394535775
247.80743179039453
202.84757708117965
264.28502394535775
247.80743179039453
268.5015576927478
257.6352234090854
248.7098899015886
239.13656204185105
259.73856079091274
267.25154807395495
232.3911233858032
259.33376455737584
264.7565106021946
238.39494370189314
233.1847124230499
235.3044809750323
247.24760141657592
261.46733251992794
257.3458961713786
246.84546840254006
216.69131045866015
258.63810282725717
249.83493268681417
263.1391119427071
258.42895909707954
259.1831246464867
255.40143158014754
245.4802081537

In [ ]:
# height = 180
# ape_index = 1.02
# flexibility = 9
# strength = 117
# weight = 70

# reach = height * ape_index 
# flex_bonus = (flexibility/10)*0.15
# strength_bonus = (strength/100)*0.2
# weight_penalty = weight*0.05
# max_reach = reach * (1 + strength_bonus + flex_bonus) - weight_penalty

# max_reach

247.8484